# <center>PPP Loans</center>

A *paycheck protection program* is a Small Business Administration (SBA) backed loan that helps businesses keep their workforce employed during the COVID-19 pandemic.

The PPP is designed to provide a direct incentive for small businesses to keep their workers on payroll.
***

## Types of PPP

####  First Draw PPP

* These can be used to help fund payroll costs, including benefits. Can be used to pay mortgage interest, rent, utilities, worker production costs, property damage, and supplier costs.

* Borrowers can qualify for loan forgiveness if during the 8-24 week covered period following disbursement:
    * employee/compensation levels are maintained
    * loan proceeds are used on payroll costs and othe eligible
      expenses
    * 60% or more of the proceeds are spent on payroll costs

#### Second Draw PPP

* For those who have previously recieved a PPP loan. This has the same loan terms as the *first draw PPP*.

* The loan forgiveness qualiofications are the same as those listed in the *first draw PPP*.

***

# Fraud within PPP

* *PPP fraud* is the abuse of a loan program for the pruppsing of 'cushioning' the blow for millions of Americans.

* We can also define it as when a business submits false information when filling out a PPP loan application or when attempting to certify PPP loan forgiveness.

## Ways to commit PPP fraud

Other than make false claims, ways to commit PPP fraud include:
* Scammers commit *loan stacking* by applying from multiple lenders
* Using the loan proceeds for improper or unapproved purposes
* providing false statements during loan audits or fraud investigations

According to the SBA, the msot common types of loan fraud are:
* **Loan agent fraud**:
    * a borrower or lender pays a loan agent to prepare documentaion for a SBA loan application and refer the borrower to a lender. 
* **Borrower fraud**:
    * they may intentionally provide false information to lending institutions during the application process.

Other ways to find fraud bsaed off data can be found here: https://www.linkedin.com/pulse/7-easy-ways-find-fraud-your-ppp-loan-portfolio-based-off-nichols

***

#

# <center>Using Neo4j to Load Data</center>

### Connect to the database

In [1]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience
import pandas as pd

uri = "neo4j://20.242.190.214:7687"
user = "neo4j"
password = "neo4j101$"

gds = GraphDataScience(uri, auth=(user,password))
gds.set_database("neo4j")

gds.run_cypher("""

MATCH (n) RETURN id(n)
LIMIT 1

""")

,id(n)


### Data Elements Available in CSV File

In [ ]:
gds.run_cypher("""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row RETURN * LIMIT 1

""")

#

# <center>Loading the Data</center>

### Borrower Nodes

In [2]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{
    WITH row
    CREATE (:Borrower {{BorrowerName: row.BorrowerName, RuralUrbanIndicator: row.RuralUrbanIndicator, BusinessAgeDescription: row.BusinessAgeDescription, JobsReported: row.JobsReported, NAICSCode: row.NAICSCode, Race: row.Race, PAYROLL_PROCEED: row.PAYROLL_PROCEED, MORTGAGE_INTEREST_PROCEED: row.MORTGAGE_INTEREST_PROCEED, RENT_PROCEED: row.RENT_PROCEED, BusinessType: row.BusinessType, Gender: row.Gender, Veteran: row.Veteran, NonProfit: row.NonProfit}})
}} IN TRANSACTIONS OF 10000 ROWS

""")

ClientError: {code: Neo.ClientError.Statement.ExternalResourceFailed} {message: Couldn't load the external resource at: file:/var/lib/neo4j/import/PPP_Dataset.csv (Transactions committed: 0)}

### Loan Nodes

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{
    WITH row
    MERGE (:Loan {{LoanNumber: toInteger(row.LoanNumber), CurrentApprovalAmount: toFloat(row.CurrentApprovalAmount), LoanStatus: row.LoanStatus, InitialApprovalAmount: toFloat(row.InitialApprovalAmount), Loan_Outstanding_Am: toFloat(row.Loan_Outstanding_Am), ForgivenessAmount: toFloat(row.ForgivenessAmount)}})
}} IN TRANSACTIONS OF 10000 ROWS

""")

### Project Nodes

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{
    WITH row
    CREATE (:Project {{ProjectCity: row.ProjectCity, ProjectCountyName: row.ProjectCountyName, ProjectState: row.ProjectState, ProjectZip: toInteger(row.ProjectZip)}})
}} IN TRANSACTIONS OF 10000 ROWS

""")

### Proceeds Nodes

In [6]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{
    WITH row
    CREATE(:Proceeds {{utilities: row.utilities, payroll: row.payroll, mortgageInterest: row.mortgageInterest, rent: row.rent, refinanceEidl: row.refinanceEidl, healthCare: row.healthCare, debtInterest: row.debtInterest}})
}} IN TRANSACTIONS OF 10000 ROWS

""")

Failed to read from defunct connection ResolvedIPv4Address(('20.242.190.214', 7687)) (IPv4Address(('20.242.190.214', 7687)))
Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information

### LoanStatus Nodes

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

CREATE(:LoanStatus {{date: date(row.date), exempt: toBoolean(row.exempt), PaidInFull: toBoolean(row.PaidInFull), chargedOff: toBoolean(row.chargedOff)}})

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Lender Servicing Nodes

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

CREATE(:LenderServicing {{locationId: row.locationId, name: row.name}})

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Lender Originating Nodes

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

CREATE(:LenderOriginating {{locationId: row.locationId, name: row.name}})

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Demographics Nodes

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

CREATE(:Demographics {{gender: row.gender, veteran: toBoolean(row.veteran), race: row.race}})

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Address Borrower Nodes

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

CREATE(:AddressBorrower {{street: row.street, city: row.city, state: row.state, zip: row.zip}})

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Address of Lender Servicing Nodes

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

CREATE(:AddressLenderServicing {{street: row.street, city: row.city, state: row.state, zip: row.zip}})

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Address of Lender Originating Nodes

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

CREATE(:AddressLenderOriginating {{street: row.street, city: row.city, state: row.state, zip: row.zip}})

}} IN TRANSACTIONS OF 10000 ROWS

""")

#

# <center>Indexing</center>

### BorrowerName (Borrower)

In [ ]:
gds.run_cypher("""

CREATE INDEX Borrower_name_index IF NOT EXISTS
FOR (x:Borrower) ON (x.BorrowerName)

""")

### LoanNumber (Loan)

In [4]:
gds.run_cypher(f"""

CREATE INDEX Loan_number_index IF NOT EXISTS FOR (x:Loan)
ON (x.LoanNumber)

""")

""


### ProjectCity (Project)

In [5]:
gds.run_cypher("""

CREATE INDEX Project_city_index IF NOT EXISTS
FOR (x:Project) ON (x.ProjectCity)

""")

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '': expected "FOR" (line 4, column 2 (offset: 49))
"CREATE INDEX Project_city_index IF NOT EXISTS"
                                              ^}

### Payroll (Proceeds)

In [ ]:
gds.run_cypher("""

CREATE INDEX Proceeds_payroll_index IF NOT EXISTS
FOR (x:Proceeds) ON (x.payroll)

""")

### Date (LoanStatus)

In [ ]:
gds.run_cypher("""

CREATE INDEX LoanStatus_date_index IF NOT EXISTS
FOR (x:LoanStatus) ON (x.date)

""")

### Location ID (LenderServicing, LenderOriginating)

In [7]:
gds.run_cypher("""

CREATE INDEX LenderServicing_locationId_index IF NOT EXISTS
FOR (x:LenderServicing) ON (x.locationId)

""")

Failed to read from defunct connection ResolvedIPv4Address(('20.242.190.214', 7687)) (IPv4Address(('20.242.190.214', 7687)))
Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information

In [ ]:
gds.run_cypher("""

CREATE INDEX LenderOriginating_locationId_index IF NOT EXISTS
FOR (x:LenderOriginating) ON (x.locationId)

""")

### Race (Demographics)

In [ ]:
gds.run_cypher("""

CREATE INDEX Demographics_race_index IF NOT EXISTS
FOR (x:Demographics) ON (x.race)

""")

### Zip (Borrower, AddressLenderServicing, AddressLenderOriginating)

In [ ]:
gds.run_cypher("""

CREATE INDEX AddressBorrower_zip_index IF NOT EXISTS
FOR (x:AddressBorrower) ON (x.zip)

""")

In [8]:
gds.run_cypher("""

CREATE INDEX AddressLenderServicing_zip_index IF NOT EXISTS
FOR (x:AddressLenderServicing) ON (x.zip)

""")

""


In [ ]:
gds.run_cypher("""

CREATE INDEX AddressLenderOriginating_zip_index IF NOT EXISTS
FOR (x:AddressLenderOriginating) ON (x.zip)

""")

#

# <center>Creating the Relationships</center>

### Borrower -- [:BORROWED] --> Loan

In [4]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

//Node
MATCH (source:Borrower)
    WHERE source.BorrowerName = (row.BorrowerName)

//Target Node
MATCH (target:Loan)
    WHERE target.LoanNumber = toInteger(row.LoanNumber)

//Edges
CREATE (source)-[r:BORROWED]->(target)

}} IN TRANSACTIONS OF 1000 ROWS

""")

DatabaseError: {code: Neo.DatabaseError.Statement.ExecutionFailed} {message: Java heap space}

### Loan -- [:RELATED_TO] --> Project

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

//Node
MATCH (source:Loan)
    WHERE source.LoanNumber = toInteger(row.LoanNumber)

//Target Node
MATCH (target:Project)
    WHERE target.ProjectCity = row.ProjectCity

//Edges
CREATE (source)-[r:RELATED_TO]->(target)

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Loan -- [:HAS_PROCEEDS] --> Proceeds

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

//Node
MATCH (source:Loan)
    WHERE source.LoanNumber = toInteger(row.LoanNumber)

//Target Node
MATCH (target:Proceeds)
    WHERE target.payroll = row.payroll

//Edges
CREATE (source)-[r:HAS_PROCEEDS]->(target)

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Loan -- [:HAS_STATUS] --> LoanStatus

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

//Node
MATCH (source:Loan)
    WHERE source.LoanNumber = toInteger(row.LoanNumber)

//Target Node
MATCH (target:LoanStatus)
    WHERE target.date = date(row.date)

//Edges
CREATE (source)-[r:HAS_STATUS]->(target)

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Loan -- [:SERVICED_BY] --> LenderServicing

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

//Node
MATCH (source:Loan)
    WHERE source.LoanNumber = toInteger(row.LoanNumber)

//Target Node
MATCH (target:LenderServicing)
    WHERE target.locationId = row.locationId

//Edges
CREATE (source)-[r:SERVICED_BY]->(target)

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Loan -- [:ORIGINATED_AT] --> LenderOriginating

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

//Node
MATCH (source:Loan)
    WHERE source.LoanNumber = toInteger(row.LoanNumber)

//Target Node
MATCH (target:LenderOriginating)
    WHERE target.locationId = row.locationId

//Edges
CREATE (source)-[r:ORIGINATED_AT]->(target)

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Borrower -- [:DEMOGRAPHICS] --> Demographics

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

//Node
MATCH (source:Borrower)
    WHERE source.BorrowerName = row.BorrowerName

//Target Node
MATCH (target:Demographics)
    WHERE target.race = row.race

//Edges
CREATE (source)-[r:DEMOGRAPHICS]->(target)

}} IN TRANSACTIONS OF 10000 ROWS

""")

### Borrower -- [:HAS_ADDRESS] --> AddressBorrower

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

//Node
MATCH (source:Borrower)
    WHERE source.BorrowerName = row.BorrowerName

//Target Node
MATCH (target:AddressBorrower)
    WHERE target.zip = row.zip

//Edges
CREATE (source)-[r:HAS_ADDRESS]->(target)

}} IN TRANSACTIONS OF 10000 ROWS

""")

### LenderOriginating -- [:HAS_ADDRESS] --> AddressLenderOriginating

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

//Node
MATCH (source:LenderOriginating)
    WHERE source.locationId = row.locationId

//Target Node
MATCH (target:AddressLenderOriginating)
    WHERE target.zip = row.zip

//Edges
CREATE (source)-[r:HAS_ADDRESS]->(target)

}} IN TRANSACTIONS OF 10000 ROWS

""")

### LenderServicing -- [:HAS_ADDRESS] --> AddressLenderServicing

In [ ]:
gds.run_cypher(f"""

LOAD CSV WITH HEADERS from "file:///PPP_Dataset.csv" as row
CALL {{

WITH row

//Node
MATCH (source:LenderServicing)
    WHERE source.locationId = row.locationId

//Target Node
MATCH (target:AddressLenderServicing)
    WHERE target.zip = row.zip

//Edges
CREATE (source)-[r:HAS_ADDRESS]->(target)

}} IN TRANSACTIONS OF 10000 ROWS

""")